In [0]:
import requests
from bs4 import BeautifulSoup
from urllib2 import urlopen
import pandas as pd

id_list = (1000002,
 1000004,
 1000005,
 1000020,
 1000035,
 1000046,
 1000047,
 1000049,
 1000063,
 1000072,
 1000080,
 1000094,
 1000097,
 1000120,
 1000123,
 1000124,
 1000125,
 1000126,
 1000127,
 1000128,
 1000129,
 1000130,
 1000134,
 1000135,
 1000144,
 1000151,
 1000154,
 1000160,
 1000161,
 1000162,
 1000179,
 1000180,
 1000181,
 1000182)

In [0]:
df = pd.DataFrame()

for id in id_list:
  user_id = id
  url = 'http://lpstock.ru/corp.php?corpid=%d' % (user_id) # url для второй страницы
  html_doc = urlopen(url).read()
  soup = BeautifulSoup(html_doc)
  for x in soup.find_all("tr"):
    title = soup.title.text
    tmp = x
    try:
      if x.text[2:14] != 'Amarr Empire':
        tmp_dict = {
        "Fraction":title[:-10],
        "Name":tmp.contents[1].text.split('\n')[0].encode('utf-8'),
        "ItemCount":tmp.contents[3].text,
        "LP":tmp.contents[5].attrs.get('abbr'),
        "ISK":tmp.contents[7].attrs.get('abbr'),
        "Spendings":tmp.contents[9].attrs.get('abbr'),
        "SellPrice":tmp.contents[11].attrs.get('abbr'),
        "Margin_LP":tmp.contents[13].contents[0]}
        df = df.append(tmp_dict,ignore_index=True)
      else:
        pass
    except:
#       print x.text
      pass

df = df[df.SellPrice.isnull() != True]

In [0]:
import re
for col in ('ISK','ItemCount','LP','Margin_LP','SellPrice','Spendings'):
    df[col] = df[col].astype('str').apply(lambda x:float(re.sub(u'[^0-9-.]', '', x.encode('utf-8'))))

In [0]:
# print(df.info())

In [0]:
# for obj in df.Fraction.unique():
#     print obj.replace(' [LPStock]','')

In [0]:
# df[df.Name == 'Caldari Navy Ballistic Control System Blueprint']

In [0]:
import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO

import pandas as pd

TESTDATA=StringIO("""type	ratio	Fraction
FW	0	Amarr Empire: 24th Imperial Crusade
Empire	0.8	Amarr Empire: Amarr Constructions
Empire	0.8	Amarr Empire: Imperial Shipment
Empire	0.8	Amarr Empire: Ministry of War
Empire	0.8	Ammatar Mandate: Ammatar Consulate
Empire	0.8	Ammatar Mandate: Ammatar Fleet
Empire	0.8	Ammatar Mandate: Nefantar Miner Association
Pirate	0	Angel Cartel: Archangels
Pirate	0	Blood Raider Covenant: Blood Raiders
Empire	0.8	Caldari State: Caldari Navy
Empire	0.8	Caldari State: CBD Corporation
Empire	0.8	Caldari State: Hyasyoda Corporation
Empire	0.8	Caldari State: Lai Dai Corporation
FW	0	Caldari State: State Protectorate
Empire	0.8	Caldari State: Ytiri
Concord	1	CONCORD Assembly: CONCORD
FW	0	Gallente Federation: Federal Defense Union
Empire	0.8	Gallente Federation: Federation Navy
Empire	0.8	Gallente Federation: Material Acquisition
Empire	0.8	Gallente Federation: TransStellar Shipping
Pirate	0	Guristas Pirates: Guristas
Empire	0.8	Khanid Kingdom: Khanid Innovation
Empire	0.8	Minmatar Republic: Brutor Tribe
Empire	0.8	Minmatar Republic: Krusual Tribe
Empire	0.8	Minmatar Republic: Sebiestor Tribe
FW	0	Minmatar Republic: Tribal Liberation Force
Pirate	0	Mordu's Legion Command: Mordu's Legion
Outsiders	0.4	ORE: Outer Ring Excavations
Pirate	0	Sansha's Nation: True Creations
Pirate	0	Sansha's Nation: True Power
Pirate	0	Serpentis: Serpentis Corporation
Outsiders	0.4	Servant Sisters of EVE: Sisters of EVE
Outsiders	0.4	The Syndicate: Intaki Bank
Outsiders	0.4	Thukker Tribe: Thukker Mix""")

fractions = pd.read_csv(TESTDATA, sep="\t")

In [0]:
df_2 = pd.merge(df, fractions, how='left', on=['Fraction'])

In [0]:
df_2['Margin_Concord_LP'] = df_2.Margin_LP * df_2.ratio

In [0]:
df_2 = df_2.sort_values(['Fraction','Name']).drop_duplicates(subset=['Name']).reset_index(drop=True).sort_values('Margin_Concord_LP',ascending=False)

In [0]:
# df_2[df_2.type != 'Outsiders'].head(30).astype('str')

In [14]:
df_2[df_2.Name == "Inherent Implants 'Highwall' Mining MX-1005"]

,Fraction,ISK,ItemCount,LP,Margin_LP,Name,SellPrice,Spendings,type,ratio,Margin_Concord_LP
388,Amarr Empire: Amarr Constructions,31750000.0,1.0,79375.0,993.39,Inherent Implants 'Highwall' Mining MX-1005,1.106000e+08,31750000.0,Empire,0.8,794.712
